In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
sc = pyspark.SparkContext()
temp_dir = working_dir = os.path.join(os.environ["HAIL_HOME"], "tmp")
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

/opt/sanger.ac.uk/hgi/anaconda3/lib/python3.7/site-packages/hail/context.py:71: UserWarning: pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jar
  'pip-installed Hail requires additional configuration options in Spark referring\n'
Running on Apache Spark version 2.4.3
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.31-6060f9c971cc
LOGGING: writing to /opt/sanger.ac.uk/hgi/hail/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20200812-1258-0.2.31-6060f9c971cc.log


Loading BokehJS ...

In [21]:
mt= hl.read_matrix_table(f"{temp_dir}/ddd-elgh-ukbb/new_labels/chr1_chr20_ldpruned_updated.mt")
mt_with_sampleqc = hl.sample_qc(mt, name='sample_qc')
mt_with_sampleqc=mt_with_sampleqc.annotate_cols(sample_qc=mt_with_sampleqc.sample_qc.annotate(heterozygosity_rate=mt_with_sampleqc.sample_qc.n_het/mt_with_sampleqc.sample_qc.n_called))


In [2]:
pop_ht = hl.read_table(f"{tmp_dir}/ddd-elgh-ukbb/mt_pops_superpops_sampleqc.ht")

In [4]:
pop_ht.show()

s,cohort,f_stat,is_female,sex,data_type,original_pop,known_pop,superpopulation,gVCF,assigned_pop,assigned_superpop,sample_qc.dp_stats.mean,sample_qc.dp_stats.stdev,sample_qc.dp_stats.min,sample_qc.dp_stats.max,sample_qc.gq_stats.mean,sample_qc.gq_stats.stdev,sample_qc.gq_stats.min,sample_qc.gq_stats.max,sample_qc.call_rate,sample_qc.n_called,sample_qc.n_not_called,sample_qc.n_filtered,sample_qc.n_hom_ref,sample_qc.n_het,sample_qc.n_hom_var,sample_qc.n_non_ref,sample_qc.n_singleton,sample_qc.n_snp,sample_qc.n_insertion,sample_qc.n_deletion,sample_qc.n_transition,sample_qc.n_transversion,sample_qc.n_star,sample_qc.r_ti_tv,sample_qc.r_het_hom_var,sample_qc.r_insertion_deletion,sample_qc.heterozygosity_rate
str,str,float64,bool,str,str,str,str,str,str,str,str,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float32
"""EGAN00001006259""","""UK_10K_CHD""",1.40e-01,true,"""female""","""exomes""","""""","""""","""""","""1006259""","""Pakistani""","""SAS""",6.41e+01,6.65e+01,2.00e+00,7.46e+02,9.17e+01,1.97e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,517,290,181,471,0,652,0,0,502,150,0,3.35e+00,1.60e+00,NA,2.94e-01
"""EGAN00001006260""","""UK_10K_CHD""",8.57e-01,false,"""male""","""exomes""","""""","""""","""""","""1006260""","""Pakistani""","""SAS""",6.95e+01,7.10e+01,4.00e+00,5.44e+02,9.34e+01,1.69e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,508,307,173,480,0,653,0,0,510,143,0,3.57e+00,1.77e+00,NA,3.11e-01
"""EGAN00001006261""","""UK_10K_CHD""",-2.81e-02,true,"""female""","""exomes""","""""","""""","""""","""1006261""","""oth""","""SAS""",6.49e+01,6.31e+01,1.00e+00,5.61e+02,9.30e+01,1.77e+01,3.00e+00,9.90e+01,9.99e-01,987,1,0,530,264,193,457,0,650,0,0,509,141,0,3.61e+00,1.37e+00,NA,2.67e-01
"""EGAN00001006263""","""UK_10K_CHD""",8.96e-01,false,"""male""","""exomes""","""""","""""","""""","""1006263""","""Pakistani""","""SAS""",6.39e+01,6.08e+01,3.00e+00,5.28e+02,9.29e+01,1.77e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,518,273,197,470,0,667,0,0,518,149,0,3.48e+00,1.39e+00,NA,2.76e-01
"""EGAN00001006264""","""UK_10K_CHD""",7.99e-01,false,"""male""","""exomes""","""""","""""","""""","""1006264""","""Pakistani""","""SAS""",5.83e+01,5.39e+01,1.00e+00,4.80e+02,9.10e+01,1.95e+01,3.00e+00,9.90e+01,1.00e+00,988,0,0,526,288,174,462,0,636,0,0,507,129,0,3.93e+00,1.66e+00,NA,2.91e-01
"""EGAN00001006265""","""UK_10K_CHD""",-4.57e-02,true,"""female""","""exomes""","""""","""""","""""","""1006265""","""British""","""EUR""",5.57e+01,4.98e+01,2.00e+00,3.53e+02,9.10e+01,2.03e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,523,302,163,465,0,628,0,0,498,130,0,3.83e+00,1.85e+00,NA,3.06e-01
"""EGAN00001006266""","""UK_10K_CHD""",-1.28e-01,true,"""female""","""exomes""","""""","""""","""""","""1006266""","""British""","""EUR""",6.21e+01,5.59e+01,4.00e+00,4.04e+02,9.27e+01,1.80e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,522,309,157,466,0,623,0,0,487,136,0,3.58e+00,1.97e+00,NA,3.13e-01
"""EGAN00001006267""","""UK_10K_CHD""",8.29e-01,false,"""male""","""exomes""","""""","""""","""""","""1006267""","""British""","""EUR""",6.49e+01,6.44e+01,3.00e+00,7.49e+02,9.36e+01,1.68e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,538,298,152,450,0,602,0,0,470,132,0,3.56e+00,1.96e+00,NA,3.02e-01
"""EGAN00001006268""","""UK_10K_CHD""",-1.19e-01,true,"""female""","""exomes""","""""","""""","""""","""1006268""","""British""","""EUR""",6.67e+01,6.36e+01,3.00e+00,5.01e+02,9.34e+01,1.73e+01,0.00e+00,9.90e+01,1.00e+00,988,0,0,516,306,166,472,0,638,0,0,494,144,0,3.43e+00,1.84e+00,NA,3.10e-01


In [22]:
mt_with_sampleqc.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'cohort': str
    'f_stat': float64
    'is_female': bool
    'sex': str
    'data_type': str
    'original_pop': str
    'known_pop': str
    'superpopulation': str
    'gVCF': str
    'sample_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
  

In [2]:
#Read mt and run sample_qc
#mt = hl.read_matrix_table(f"{tmp_dir}/ddd-elgh-ukbb/chr1_chr20_XY_ldpruned_updated.mt")
#mt_with_sampleqc = hl.sample_qc(mt, name='sample_qc')
#mt = mt_with_sampleqc.checkpoint(f"{tmp_dir}/ddd-elgh-ukbb/Sanger_cohort_AKT_elgh_labels_sampleqc.mt", overwrite=True)

In [4]:
pca_scores=hl.read_table(f"{temp_dir}/ddd-elgh-ukbb/new_labels/pop_assignments_updated_august2020.ht")
mt= hl.read_matrix_table(f"{temp_dir}/ddd-elgh-ukbb/new_labels/chr1_chr20_ldpruned_updated.mt")
pca_scores=pca_scores.annotate(cohort=mt.cols()[pca_scores.s].cohort)

2020-08-11 15:33:44 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [5]:
pca_scores.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'known_pop': str 
    'pca_scores': array<float64> 
    'pop': str 
    'prob_African': float64 
    'prob_Bangladeshi': float64 
    'prob_British': float64 
    'prob_Caribbean': float64 
    'prob_Chinese': float64 
    'prob_Indian': float64 
    'prob_Irish': float64 
    'prob_Pakistani': float64 
    'prob_White European': float64 
    'cohort': str 
----------------------------------------
Key: ['s']
----------------------------------------


In [ ]:
pca_scores=pca_scores.annotate(cohort=mt.cols()[pca_scores.s].cohort)


In [7]:
p1=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[1], xlabel="PC1", ylabel="PC2", label=pca_scores.pop)
p2=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[2], xlabel="PC1", ylabel="PC3", label=pca_scores.pop)


show(bokeh.layouts.gridplot([[p1],[p2]]))